# Build AI PC

Read this article for instruction to build your own AI PC [Run LLMs Natively in your Java Application
](https://www.linkedin.com/pulse/run-llms-natively-your-java-application-yi-leng-yao-brpnc/?trackingId=gwi8eydkQselga2eSd8N7Q%3D%3D)

# SSH into Ubuntu Server
On the Ubuntu terminal and install openssh
```
apt update
sudo apt install openssh-server
```

To validate ssh is running
```
sudo systemctl status ssh
```

Tell firewall to allow ssh
```
sudo ufw allow ssh
```

Get local network IP address of Ubuntu machine
```
ip a
```

No on your Mac type in the command to ssh into the machine.
```
ssh <username>@<ubuntu local network ip address>
```

# TailScale Setup
TailScale is a peer-to-peer vpn that allows your computers to communicate over the internet.

Download and install Tailscale on mac from https://tailscale.com/download/.

SSH into Ubuntu and run
```
curl -fsSL https://tailscale.com/install.sh | sh
```

After setting up TailScale you can get the IP addresses of your devices in https://login.tailscale.com/admin/machines.

# Ollama Setup
Created systemd override for persistent configuration
/etc/systemd/system/ollama.service.d/override.conf:
```
[Service]
Environment="OLLAMA_HOST=0.0.0.0:11434"
```

Created Ollama config file
HOME/.ollama/config.json:
```
{
  "host": "0.0.0.0:11434",
  "origins": ["*"]
}
```
This configured Ollama to:

Listen on all network interfaces (0.0.0.0)
Accept connections from any origin
Persist settings across restarts

## Allowing Access through our firewall
Now using ufw to allow traffic to the ports 8080 and 11434 in our firewall. Run the following in Ubuntu.
```
sudo ufw allow in on tailscale0 to any port 8080
sudo ufw allow in on tailscale0 to any port 11434
```

## Start Ollama Server and Pull LLM Model
Starting Ollama Server
```
sudo systemctl start ollama

#or
sudo OLLAMA_HOST="0.0.0.0:11434" ollama serve
```

Pulling the QWEN 32b model
```
ollama pull qwen:32b
```

## Accessing Ollama on your mac
you can access Ollama on your mac through 
```
localhost 11434
```

# Jupyter Integration
Install the `Pretzel plugin`.

Go to `settings` -> `Pretzel AI Settings`

Set the base url to
```
http://localhost:11434
```

And model and copilot model to
```
qwen:32b
```

<img src="JupyterIntegration.png">

# VS Code Setup

## SSH
Install Open Remote - SSH plugin

## Code Generation
Install the Continue - Codestral, Claude, and more plugin

Then add you local model to the `.continue/config.json` file
```
 "models": [
    {
      "title": "Qwen",
      "provider": "ollama",
      "model": "qwen:32b"
    },
```

<img src="vs-code-continuedev-config.png">

# NeoVim Development
Add the [remote-nvim](https://github.com/amitds1997/remote-nvim.nvim) plugin.

Create a file named `remote-nvin.lua` in your `~/.config/nvim/lua/plugins` directory and add the following code.
```
return {
  {
    "amitds1997/remote-nvim.nvim",
  version = "*", -- Pin to GitHub releases
  dependencies = {
      "nvim-lua/plenary.nvim", -- For standard functions
      "MunifTanjim/nui.nvim", -- To build the plugin UI
      "nvim-telescope/telescope.nvim", -- For picking b/w different remote methods
  },
  config = true,
  },
}
```

Start up your neovim editor
```
nvim
```

And enter in the command to ssh into your remote server.
```
:RemoteStart
```


# Java Development
Add the following configuration to your `application.properties` file
```
# In application.properties
server.address=0.0.0.0
server.port=8080
server.tomcat.protocol-header=x-forwarded-proto
server.tomcat.remote-ip-header=x-forwarded-for
server.tomcat.use-relative-redirects=true
```

And when you spin up your Spring Boot server on Ubuntu, you can access the endpoint through `htto://localhost:8080`.